In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

main_df = pd.read_excel('ShulichIgniteData/ShulichIgniteData.xlsx', sheet_name=0, usecols="C,E,G,K,N,Q,T,W,Z")
main_df['Total Followers'] = main_df.sum(axis=1,numeric_only=True)

main_df

,Region of Focus,Entity owner (English),Parent entity (English),X (Twitter) Follower #,Facebook Follower #,Instagram Follower #,Threads Follower #,YouTube Subscriber #,TikTok Subscriber #,Total Followers
0,Anglosphere,China Media Group (CMG),Central Publicity Department,1678.0,1387432.0,9507.0,197.0,4680.0,660.0,1404154.0
1,Anglosphere,China Media Group (CMG),Central Publicity Department,2488.0,NaN,635.0,NaN,NaN,NaN,3123.0
2,China,All-China Students' Federation,Central Committee of the Communist Youth League,5136.0,NaN,NaN,NaN,NaN,NaN,5136.0
3,Afghanistan,China Media Group (CMG),Central Publicity Department,1800.0,1600000.0,NaN,NaN,695.0,326100.0,1928595.0
4,Anglosphere,People's Daily Press,Central Committee of the Chinese Communist Party,2821.0,NaN,NaN,NaN,NaN,NaN,2821.0
...,...,...,...,...,...,...,...,...,...,...
753,Singapore,Ministry of Foreign Affairs,Ministry of Foreign Affairs,NaN,1700.0,NaN,NaN,NaN,NaN,1700.0
754,Hong Kong,Office of the Commissioner of the Ministry of ...,Ministry of Foreign Affairs,NaN,853000.0,NaN,NaN,NaN,NaN,853000.0
755,Yunnan,Wenshan Radio & Television Station,Wenshan Municipal People's Government,NaN,15.0,NaN,NaN,NaN,NaN,15.0
756,Anglosphere,Xinhua News Agency,State Council,NaN,2300000.0,NaN,NaN,NaN,NaN,2300000.0


In [39]:
rofs = main_df["Region of Focus"].value_counts()
entities = main_df["Entity owner (English)"].value_counts()
parents = main_df["Parent entity (English)"].value_counts()
print(parents)

Ministry of Foreign Affairs                         293
Central Publicity Department                        275
State Council                                        38
Central Committee of the Chinese Communist Party     35
Shanghai Municipal Committee                         13
                                                   ... 
Chengdu Municipal Committee                           1
Jiaxing Municipal Committee                           1
Ministry of Commerce (MOFCOM)                         1
Shaoxing Municipal People's Government                1
Wenshan Municipal People's Government                 1
Name: Parent entity (English), Length: 69, dtype: int64


In [27]:
class Owner:
    owner_name: str
    parent_name: str
    followers: int

    def __init__(self, owner_name: str, parent_name: str, followers: int):
        self.owner_name = owner_name
        self.parent = parent_name
        self.followers = followers

class Parent:
    parent_name: str
    entity_owners: [Owner]
    followers: int

    def __init__(self, parent_name: str, followers: int):
        self.parent_name = parent_name
        self.followers = followers
        self.entity_owners = []

class RoF:
    rof_name: str
    owners = [Owner]
    chinese_influence: int

    def __init__(self, rof_name: str, chinese_influence: int):
        self.rof_name = rof_name
        self.chinese_influence = chinese_influence
        self.parents = []


In [28]:
followers_df = main_df.groupby('Region of Focus')['Total Followers'].sum().reset_index()
followers_df

,Region of Focus,Total Followers
0,ASEAN,107600.0
1,Afghanistan,5158931.0
2,African Union,5678664.0
3,Albania,1111391.0
4,Algeria,57966.0
...,...,...
167,Yemen,69500.0
168,Yunnan,176415.0
169,Zhejiang,108536.0
170,Zimbabwe,33600.0


In [50]:
rof_to_rof_dicts = {}
for index, series in main_df.iterrows():
    rof_dict = {'rof': series['Region of Focus'], 'owner': series['Entity owner (English)'], 'parent': series['Parent entity (English)'], 'followers': series['Total Followers']}
    if series['Region of Focus'] in rof_to_rof_dicts:
        rof_to_rof_dicts[series['Region of Focus']].append(rof_dict)
    else:
        rof_to_rof_dicts[series['Region of Focus']] = [rof_dict]


In [52]:
jobj = json.dumps(rof_to_rof_dicts)
file_path = 'chinese_influence_by_rof.json'
with open(file_path, 'w') as json_file:
    json.dump(jobj, json_file, indent=2)